Author: Irsyad

In [3]:
import pandas as pd
import numpy as np
import neo4j

from tqdm import tqdm
from neo4j import GraphDatabase

from graph_create_utils import *

c:\Users\irsya\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\irsya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\irsya\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Upload DataRes Network into Neo4j

Ensure that neo4j dbmss is > 4.4.0 and contains both APOC and GDSL

**Table of Contents**

1) Connect to the Neo4j DBMS

2) Create Constraints

3) Upload Respondees as nodes

4) Match Relationships

### Connect to Neo4j

In [3]:
# Seen from :server status
uri = "bolt://localhost:7687"

# default user for graph database is neo4j
# auth = ("neo4j", "password")
auth = ("neo4j","datares")

driver = GraphDatabase.driver(uri = uri, auth = auth)
driver.verify_connectivity()

C:\Users\irsya\AppData\Local\Temp/ipykernel_17524/4225089448.py:9: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


'Neo4j/4.4.7'

### Create Constraints for Node Name and Node Relationship

In [15]:
try:
    name_query = "CREATE CONSTRAINT ON (p:Person) ASSERT p.name IS UNIQUE"       

    info = driver.session().run(name_query)
    response = driver.session().run("CALL db.constraints").data()
    print(response)

except:
    print("uniqueness constraint already exists")

    name_query = "CALL db.constraints;"
    info = driver.session().run(name_query)
    response = driver.session().run("CALL db.constraints").data()
    print(response)



uniqueness constraint already exists
[{'name': 'constraint_550b2518', 'description': 'CONSTRAINT ON ( person:Person ) ASSERT (person.name) IS UNIQUE', 'details': "Constraint( id=4, name='constraint_550b2518', type='UNIQUENESS', schema=(:Person {name}), ownedIndex=3 )"}]


### Deploy the Names as Nodes

data preprocessing with pandas

In [3]:
data = pd.read_excel("form_responses.xlsx")
data.fillna("No")
data

,Timestamp,First and Last Name:,Your DataRes Team for S22,Favorite Boba Shop,do u like datares,Do you know who Madison is?,Do you know Colin?,Raymond Bai,Karina Santoso,Drew Letvin,...,Terry Ming,Jordan Lin,Prateik Sinha,Allen Tong,Chris Apton,Unnamed: 127,Unnamed: 128,Brandon Zhao,Jim Liu,Anvesha Dutta
0,2022-04-11 17:11:59.618,Charisse Hung,UCLA Athletics,Sunright (my favorite thats why its at top),fuck yea,yup super cool,literally a genius,Yes,Yes,Yes,...,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN
1,2022-04-11 17:12:02.940,Jason Clark,UCLA Athletics,Sharetea,fuck yea,yup super cool,literally a genius,Yes,Yes,Yes,...,No,No,No,Yes,No,NaN,NaN,NaN,NaN,NaN
2,2022-04-11 17:12:10.356,Karina Santoso,UCLA Athletics,Sunright (my favorite thats why its at top),fuck yea,yup super cool,literally a genius,Yes,Yes,Yes,...,No,Yes,No,Yes,No,NaN,NaN,NaN,NaN,NaN
3,2022-04-11 17:12:10.471,Raymond Bai,UCLA Athletics,Sharetea,fuck yea,yup super cool,literally a genius,Yes,Yes,Yes,...,No,No,No,Yes,No,NaN,NaN,NaN,NaN,NaN
4,2022-04-11 17:12:21.225,Oliver de Visser,UCLA Athletics,Teaspoon,fuck yea,no :(,no :(,Yes,Yes,Yes,...,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2022-04-19 18:16:17.190,Samantha Chung,Data Blog,Yifang,fuck yea,yup super cool,no :(,Yes,No,No,...,No,No,No,No,No,NaN,NaN,No,No,No
58,2022-04-19 18:16:56.025,Kowoon Jeong,Data Blog,Sharetea,fuck yea,yup super cool,no :(,No,No,No,...,No,No,No,No,No,NaN,NaN,No,No,No
59,2022-04-19 19:10:06.108,Megha Velakacharla,Data Blog,Sharetea,fuck yea,yup super cool,no :(,No,No,No,...,No,No,No,No,No,NaN,NaN,No,Yes,No
60,2022-05-03 13:04:48.204,Madison Kohls,Co-President (yes i added a label for u colin ...,Sunright (my favorite thats why its at top),fuck yea,yup super cool,literally a genius,Yes,Yes,Yes,...,Yes,Yes,No,Yes,Yes,NaN,NaN,Yes,Yes,Yes


In [4]:
names = data["First and Last Name:"].dropna()
names.to_list()

names

0          Charisse Hung
1            Jason Clark
2         Karina Santoso
3            Raymond Bai
4       Oliver de Visser
             ...        
59        Samantha Chung
60         Kowoon Jeong 
61    Megha Velakacharla
62         Madison Kohls
63          Colin Curtis
Name: First and Last Name:, Length: 62, dtype: object

In [5]:
uri = "bolt://localhost:7687"
password = "datares"
auth = ("neo4j", password)
print("starting datares graph create \n")

driver = GraphDatabase.driver(uri = uri, auth = auth)
print("connecting:", driver.verify_connectivity())

#create constraint
create_constraint(driver)

data = data_preprocessing()

#upload nodes
responses = upload_nodes(data, driver)

#upload relationships
#upload_relationship(data, responses, driver)

starting datares graph create 



C:\Users\irsya\AppData\Local\Temp/ipykernel_1752/3997970849.py:7: ExperimentalWarning: The configuration may change in the future.
  print("connecting:", driver.verify_connectivity())


connecting: Neo4j/4.4.7
creating uniqueness constraint


deploying person nodes:   0%|          | 0/62 [00:00<?, ?it/s]

uniqueness constraint already exists: [{'name': 'constraint_550b2518', 'description': 'CONSTRAINT ON ( person:Person ) ASSERT (person.name) IS UNIQUE', 'details': "Constraint( id=4, name='constraint_550b2518', type='UNIQUENESS', schema=(:Person {name}), ownedIndex=3 )"}]
success



deploying person nodes: 100%|██████████| 62/62 [00:00<00:00, 413.33it/s]

success



In [6]:


#lowercase the columns
data.columns = data.columns.str.lower()

#change colin and madison to the headings
new_headings = data.columns.to_list()
new_headings[5] = "madison kohls"
new_headings[6] = 'colin curtis'
data.columns = new_headings

#names as the index
data.index = data["first and last name:"]
data.index = data.index.str.lower()

#filter out data
wrong_names = ['ziyue jin', 'tian ouyang', 'natalia f', 'shiyu murashima', 'trent bellinger', 'christine shen ', 'daniel fsng', 'kowoon jeong ']
new_headings = [item for item in responses if item not in wrong_names]
data = data[new_headings]

#filter out colin and mads data
data["colin curtis"] = data["colin curtis"].replace(to_replace = "literally a genius", value = "Yes")
data["madison kohls"] = data["madison kohls"].replace(to_replace = "yup super cool", value = "Yes")

data



,charisse hung,jason clark,karina santoso,raymond bai,oliver de visser,allen tong,joshua li,rebecca xu,deana moghaddas,audrey cabrera,...,ruyi lu,joseph ramirez,dan wu,ryan barney,david spector,steven lu chen,samantha chung,megha velakacharla,madison kohls,colin curtis
first and last name:,,,,,,,,,,,,,,,,,,,,,
charisse hung,Yes,Yes,Yes,Yes,No,No,Yes,Yes,No,Yes,...,No,No,No,No,No,No,No,No,None,None
jason clark,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,No,No,No,No,No,No,No,No,None,None
karina santoso,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,No,No,No,No,No,No,No,No,None,None
raymond bai,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,...,No,No,No,No,No,No,No,No,None,None
oliver de visser,No,No,Yes,Yes,Yes,No,No,No,No,No,...,No,No,No,No,No,No,No,No,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
samantha chung,No,No,No,Yes,No,No,No,No,No,No,...,No,No,No,No,No,No,Yes,Yes,None,None
kowoon jeong,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,No,No,None,None
megha velakacharla,No,No,No,No,No,No,No,No,No,No,...,No,No,No,No,No,Yes,Yes,Yes,None,None


In [61]:
def create_relationships(tx, name1, name2) -> None:
    """
    Args:
        id1 is the id of first node
        id2 is id of second 
        NOTE: SRC-->DEST
    """
    query = """
            MATCH (p:Person {name: $name1})
            MATCH (n:Person {name: $name2})
            MERGE (p)-[:KNOWS]-(n)
            """
    tx.run(query, name1 = name1, name2 = name2)

In [66]:
for column_name in tqdm(data.columns, desc = "deploying relationships"):
    for i in data.index:
        if data.loc[i, column_name] == "Yes":
            if i != column_name:
                driver.session().write_transaction(create_relationships, i, column_name)


deploying relationships: 100%|██████████| 54/54 [00:02<00:00, 19.38it/s]


In [65]:
len(ls)

408

In [60]:
len(ls)

408